In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

%matplotlib inline

In [94]:
members = pd.read_csv('../Data/NSS_GreatCare_Members.csv', parse_dates=[61,62])
admissions = pd.read_csv('../Data/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12])

C:\Users\rache\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


What is our 30-day all cause readmission rate across hospitals our members are admitted to?

In [95]:
print(admissions['Encounter End Date'])

0      2013-01-14
1      2014-01-12
2      2012-06-21
3      2012-12-09
4      2015-03-14
          ...    
6208   2015-02-11
6209   2014-08-10
6210   2015-11-23
6211   2015-11-27
6212   2010-03-31
Name: Encounter End Date, Length: 6213, dtype: datetime64[ns]


In [96]:
admissions_dedup = admissions.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = admissions_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
admissions.shape, admissions_dedup.shape

((6213, 23), (6149, 23))

In [97]:
total_admissions = admissions.shape[0]

In [98]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        ##set first event as an intiial event or a non-qualifying readmission as initial event
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            readmission = False
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'index'
        ##check if qualifying readmission and change readmit status of most recently added initial event to true
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'readmission'
        ##check for duplicate readmission
        else:
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'duplicate readmission'
            
    return patient_index_events, patient_readmission_events

In [99]:
total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in admissions_dedup['Patient ID'].unique():
    patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        admissions.loc[admissions['Encounter ID'] == encounter_id, 'type'] = 'index'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-99-b229257dddde>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
<ipython-input-99-b229257dddde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


In [100]:
admissions['type'].value_counts()

index                    5536
readmission               527
duplicate readmission      86
Name: type, dtype: int64

What percent of patients have a PCP visit within a week of discharge?

In [110]:
total_hospital_visits = admissions_dedup.shape[0]
total_pcp_visits_in_7_days = 0

for ind, row in admissions_dedup.iterrows():
    patient = row['Patient ID']
    discharge_date = row['Encounter End Date']
    
    for i,r in pcp_visits[pcp_visits['Patient ID']==patient].iterrows():
        if r['Encounter Date']<= discharge_date + dt.timedelta(days=7) and r['Encounter Date'] >= discharge_date:
            total_pcp_visits_in_7_days += 1
            break

print('Total Hospital Visit:', total_hospital_visits)
print('Total PCP Visits in 7 Days:', total_pcp_visits_in_7_days)
print(str.format("PCP Visit Rate: {}%", round(total_pcp_visits_in_7_days/total_hospital_visits*100,2)))

Total Hospital Visit: 6149
Total PCP Visits in 7 Days: 1860
PCP Visit Rate: 30.25%
